In [1]:
import os,sys
#Change and check directory
os.chdir('C:\WGU\D209\PA\Task1')
os.getcwd()

'C:\\WGU\\D209\\PA\\Task1'

In [2]:
#import necessary libraries
import pandas as pd
#For array functionality
import numpy as np
#For preprocessing the data
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#for getting the best value for k
from sklearn.model_selection import GridSearchCV
#for the KNN model
#import sklearn
from sklearn.neighbors import KNeighborsClassifier
#for analyzing the model
from sklearn.metrics import confusion_matrix, roc_auc_score

#load the data frame
df = pd.read_csv('churn_clean.csv', index_col = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer_id           10000 non-null  object 
 1   Interaction           10000 non-null  object 
 2   UID                   10000 non-null  object 
 3   City                  10000 non-null  object 
 4   State                 10000 non-null  object 
 5   County                10000 non-null  object 
 6   Zip                   10000 non-null  int64  
 7   Lat                   10000 non-null  float64
 8   Lng                   10000 non-null  float64
 9   Population            10000 non-null  int64  
 10  Area                  10000 non-null  object 
 11  TimeZone              10000 non-null  object 
 12  Job                   10000 non-null  object 
 13  Children              10000 non-null  int64  
 14  Age                   10000 non-null  int64  
 15  Income                10

In [3]:
#Data Cleaning
#Detect and treat NULLS - we can see from the describe call that there is only one column with missing values
#InternetService: This column actually doesn't have any null values it just got read in as NULL rather than None indicating the customer doesn't have this service
df['InternetService'].fillna('None',inplace=True)

In [4]:
#Drop initial columns we won't be using
df = df.drop(columns=['Customer_id', 'Interaction', 'UID', 'City', 'State', 'County', 'Zip', 'Lat', 'Lng', 'TimeZone', 'Area', 'Job', 'Children', 'Age', 'Income', 'Marital', 'Gender', 'Techie', 'Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'])

In [5]:
#Summary Statistics and Reviewing for Outliers
df.describe()

,Population,Outage_sec_perweek,Email,Contacts,Yearly_equip_failure,Tenure,MonthlyCharge,Bandwidth_GB_Year
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,9756.562400,10.001848,12.016000,0.994200,0.398000,34.526188,172.624816,3392.341550
std,14432.698671,2.976019,3.025898,0.988466,0.635953,26.443063,42.943094,2185.294852
min,0.000000,0.099747,1.000000,0.000000,0.000000,1.000259,79.978860,155.506715
25%,738.000000,8.018214,10.000000,0.000000,0.000000,7.917694,139.979239,1236.470827
50%,2910.500000,10.018560,12.000000,1.000000,0.000000,35.430507,167.484700,3279.536903
75%,13168.000000,11.969485,14.000000,2.000000,1.000000,61.479795,200.734725,5586.141370
max,111850.000000,21.207230,23.000000,7.000000,6.000000,71.999280,290.160419,7158.981530


In [6]:
categorical_columns = ['Churn','Port_modem', 'Tablet', 'Phone', 'Multiple', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Contract', 'InternetService', 'PaymentMethod']
for x in categorical_columns:
    print(df[x].describe())

count     10000
unique        2
top          No
freq       7350
Name: Churn, dtype: object
count     10000
unique        2
top          No
freq       5166
Name: Port_modem, dtype: object
count     10000
unique        2
top          No
freq       7009
Name: Tablet, dtype: object
count     10000
unique        2
top         Yes
freq       9067
Name: Phone, dtype: object
count     10000
unique        2
top          No
freq       5392
Name: Multiple, dtype: object
count     10000
unique        2
top          No
freq       6424
Name: OnlineSecurity, dtype: object
count     10000
unique        2
top          No
freq       5494
Name: OnlineBackup, dtype: object
count     10000
unique        2
top          No
freq       5614
Name: DeviceProtection, dtype: object
count     10000
unique        2
top          No
freq       6250
Name: TechSupport, dtype: object
count     10000
unique        2
top          No
freq       5071
Name: StreamingTV, dtype: object
count     10000
unique        2
top       

In [7]:
#One hot encoding 
continuous_columns = ['Population', 'Outage_sec_perweek', 'Email', 'Contacts', 'Yearly_equip_failure', 'Tenure', 'MonthlyCharge', 'Bandwidth_GB_Year']
categorical_columns_yn = ['Churn','Port_modem', 'Tablet', 'Phone', 'Multiple', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
categorical_columns_multiple = ['Contract', 'InternetService', 'PaymentMethod']
df = pd.get_dummies(df, columns=categorical_columns_yn, drop_first=True, dtype = int)
df = pd.get_dummies(df, columns=categorical_columns_multiple, drop_first=False, dtype = int)
#Rename the columns for clarity and to remove spaces
df = df.rename(columns = {'Contract_One year':'Contract_One_Year', 'Contract_Two Year':'Contract_Two_Year','InternetService_Fiber Optic':'InternetService_Fiber_Optic', 'InternetService_Fiber Optic':'InternetService_Fiber_Optic', 'PaymentMethod_Credit Card (automatic)':'PaymentMethod_CC', 'PaymentMethod_Electronic Check':'PaymentMethod_ECheck', 'PaymentMethod_Mailed Check':'PaymentMethod_MCheck', 'PaymentMethod_Bank Transfer(automatic)':'PaymentMethod_Bank_Transfer'})
categorical_columns = ['Port_modem_Yes', 'Tablet_Yes', 'Phone_Yes', 'Multiple_Yes', 'OnlineSecurity_Yes', 'OnlineBackup_Yes', 'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes', 'StreamingMovies_Yes', 'PaperlessBilling_Yes', 'Contract_One_Year', 'Contract_Two_Year', 'InternetService_Fiber_Optic','InternetService_None','InternetService_DSL','PaymentMethod_CC','PaymentMethod_ECheck', 'PaymentMethod_MCheck', 'PaymentMethod_Bank_Transfer','Contract_Month-to-month']
#Store all the columns for easy access later
columns = categorical_columns + continuous_columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Population                   10000 non-null  int64  
 1   Outage_sec_perweek           10000 non-null  float64
 2   Email                        10000 non-null  int64  
 3   Contacts                     10000 non-null  int64  
 4   Yearly_equip_failure         10000 non-null  int64  
 5   Tenure                       10000 non-null  float64
 6   MonthlyCharge                10000 non-null  float64
 7   Bandwidth_GB_Year            10000 non-null  float64
 8   Churn_Yes                    10000 non-null  int32  
 9   Port_modem_Yes               10000 non-null  int32  
 10  Tablet_Yes                   10000 non-null  int32  
 11  Phone_Yes                    10000 non-null  int32  
 12  Multiple_Yes                 10000 non-null  int32  
 13  OnlineSecurity_Yes   

In [8]:
#Use SelectKBest to further narrow down the features to get the best ones for the model
X = df.drop('Churn_Yes', axis=1)
y = df["Churn_Yes"]
feature_names = X.columns

# Initialize the class and call fit_transform
skbest = SelectKBest(k='all') # k= features
X_new = skbest.fit_transform(X, y)
X_new.shape	

### Finding P-values to select statistically significant features 
p_values = pd.DataFrame({'Feature': X.columns, 'p_value':skbest.pvalues_}).sort_values('p_value')
p_values[p_values['p_value'] < .05]
features_to_keep = p_values['Feature'][p_values['p_value'] < .05]
# Print the name of the selected features
print(features_to_keep)

5                          Tenure
6                   MonthlyCharge
7               Bandwidth_GB_Year
17            StreamingMovies_Yes
19        Contract_Month-to-month
16                StreamingTV_Yes
21              Contract_Two_Year
20              Contract_One_Year
11                   Multiple_Yes
22            InternetService_DSL
23    InternetService_Fiber_Optic
14           DeviceProtection_Yes
13               OnlineBackup_Yes
24           InternetService_None
27           PaymentMethod_ECheck
10                      Phone_Yes
Name: Feature, dtype: object


In [9]:
df = df.drop(columns = ['PaymentMethod_Bank_Transfer','Population','Outage_sec_perweek','Email','Contacts','Tablet_Yes','Yearly_equip_failure','OnlineSecurity_Yes','PaperlessBilling_Yes','TechSupport_Yes','Port_modem_Yes','PaymentMethod_CC','PaymentMethod_MCheck'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Tenure                       10000 non-null  float64
 1   MonthlyCharge                10000 non-null  float64
 2   Bandwidth_GB_Year            10000 non-null  float64
 3   Churn_Yes                    10000 non-null  int32  
 4   Phone_Yes                    10000 non-null  int32  
 5   Multiple_Yes                 10000 non-null  int32  
 6   OnlineBackup_Yes             10000 non-null  int32  
 7   DeviceProtection_Yes         10000 non-null  int32  
 8   StreamingTV_Yes              10000 non-null  int32  
 9   StreamingMovies_Yes          10000 non-null  int32  
 10  Contract_Month-to-month      10000 non-null  int32  
 11  Contract_One_Year            10000 non-null  int32  
 12  Contract_Two_Year            10000 non-null  int32  
 13  InternetService_DSL  

In [10]:
#Scale the numeric variables 
#Initialize StandardScaler
scaler = StandardScaler()
# Apply StandardScaler to numeric columns
features = ['Tenure','MonthlyCharge', 'Bandwidth_GB_Year']
for x in features:
    df[x] = scaler.fit_transform(df[x].to_numpy().reshape(-1,1))

In [11]:
df.head()

,Tenure,MonthlyCharge,Bandwidth_GB_Year,Churn_Yes,Phone_Yes,Multiple_Yes,OnlineBackup_Yes,DeviceProtection_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_Month-to-month,Contract_One_Year,Contract_Two_Year,InternetService_DSL,InternetService_Fiber_Optic,InternetService_None,PaymentMethod_ECheck
CaseOrder,,,,,,,,,,,,,,,,,
1,-1.048746,-0.003943,-1.138487,0,1,0,1,0,0,1,0,1,0,0,1,0,0
2,-1.262001,1.630326,-1.185876,1,1,1,0,0,1,1,1,0,0,0,1,0,0
3,-0.709940,-0.295225,-0.612138,0,1,1,0,0,0,1,0,0,1,1,0,0,0
4,-0.659524,-1.226521,-0.561857,0,1,0,0,0,1,0,0,0,1,1,0,0,0
5,-1.242551,-0.528086,-1.428184,1,0,0,0,0,1,0,1,0,0,0,1,0,0


In [12]:
#Export the prepared data set
df.to_csv('PREPARED_churn_clean_data.csv')

In [13]:
# Set up the data as X and y and split the data for training and testing 
X = df.drop('Churn_Yes', axis=1)
y = df['Churn_Yes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

# Export training and test datasets
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

#transform to arrays for KNN
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [14]:
#Use grid 
param_grid = {'n_neighbors': np.arange(1, 30)}
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, param_grid)
knn_cv.fit(X_train, y_train)
knn_cv.best_params_

{'n_neighbors': 19}

In [15]:
#Fit the KNN model using grid search result of k = 19
knn = KNeighborsClassifier(n_neighbors = 19)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=19)

In [16]:
y_pred = knn.predict(X_test)
y_pred_prob = knn.predict_proba(X_test)[:,1]
c_matrix = confusion_matrix(y_test,y_pred)
print(c_matrix)

[[1362  108]
 [ 120  410]]


In [17]:
print('Model Accuracy: ', knn.score(X_test, y_test))

Model Accuracy:  0.886


In [18]:
print('Training Accuracy: ', knn.score(X_train, y_train))

Training Accuracy:  0.90125


In [19]:
print("AUC Score: ", roc_auc_score(y_test, y_pred_prob))

AUC Score:  0.9440726479270954
